<a href="https://colab.research.google.com/github/mzohaibnasir/finetuningLLM/blob/main/02_QLoRA%26LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QLoRA and LoRA Quantization
specifically used in fine tuning of LLMs.


1. **Full Parameteric Fine tuning:** all weights are updated
2. **Domain specific Fine tuning:** Finance/sales
3. **Task specific tuning:** QnA chatbots, Document QnA chatbots


In [ ]:
"https://medium.com/@mujahidabdullahi1992/an-introduction-to-lora-unpacking-the-theory-and-practical-implementation-e665c5d78295"
"https://medium.com/@yumo-bai/why-are-most-llms-decoder-only-590c903e4789"

## Full Parameteric Fine tuning:
1. need to update all weights
2. hardware resource constraints
  1. Model Monitoring
  2. Model inferencing
  3. GPU
  4. RAM



**We use LoRA & QLoRA(LoRA 2.0) to oercome these challenges.**

## LoRA & QLoRA

### What does LoRA do?
 Inatead of updating weights it tracks changes.

    Pre-trained weights + LoRA tracked weights = Fine-tuned weights


  `LoRA tracked weights:(m*n)` are of same shape as `pretrained weights:(m*n)`. These two weights will be combine to get our fine-tuned weights.

Assume `n*m is 3*3` for both matrices. Due to matrix decomposition, our `3x3 LoRA tracked weights` would be stored in two small matrices of `1x3` and `3x1`. Resultant of these both small matrix will be actual `LoRA tracked weights`. So LoRA is decomposing `LoRA weight matrix` into  two smaller matrix based on ` Rank parameter`

---

#### Matrix decomposition:
The rank of a matrix (denoted by ρ(A)) is the maximum number of linearly independent rows (or columns) in the matrix.

----
Matrix decomposition works because data delta(W) is redundant

The idea behind Low-Rank Adaptation (LoRA) is built upon the observation that the weights learned by Large Language Models (LLMs) after training often contain redundancies. Therefore, instead of fine-tuning the entire set of weights in the LLM, we can streamline the process by focusing on a low-rank approximation of the weights — essentially, a smaller set of weights that eliminates these redundancies.

According to the LoRA paper, this idea comes from the simple hypothesis that trained models have low “intrinsic rank” (simply meaning model parameters contain duplications we can do without).

`We take inspiration from Li et al. (2018a); Aghajanyan et al. (2020) which show that the learned over-parametrized models in fact reside on a low intrinsic dimension. We hypothesize that the change in weights during model adaptation also has a low “intrinsic rank”, leading to our proposed Low-Rank Adaptation (LoRA) approach`

During the fine-tuning process, all other weights remain frozen, by doing so we ensure learned weights are not altered during the finetuning process. This approach not only makes the process more efficient but also reduces the risk of overfitting and `“catastrophic forgetting”`, a phenomenon where the knowledge encapsulated in the pre-trained model is lost during the fine-tuning process.

#### Theoretical Fundamentals behind LoRA

Transitioning from a large parameter space to a low-rank approximation that is capable of introducing new information into the pre-trained Large Language Model (LLM) is a key aspect of LoRA. To understand this, we can consider the statement below from the LoRA paper.

Inspired by this, we hypothesize the updates to the weights also have a low `“intrinsic rank”` during adaptation. For a pre-trained weight matrix `Wo ∈ R d×k` , we constrain its update by representing the latter with a low-rank decomposition
    
    `Wo + ∆W = Wo + BA`,
    where
      `B ∈ R d×r` ,
      `A ∈ R r×k` ,
    and the rank
     `r
 << min(d, k)`.

The following equation succinctly captures the LoRA-based fine-tuning process:


    W = Wo + ∆W = Wo + BA

    In this equation,

    1. Wo denotes the pre-trained parameter weights
    2. ∆W denotes the learned weights to be used in adjusting the original weights
    3. W is the final fine-tuned weight that will be used during inference
    4. B is a matrix of dimension d×r and A is a matrix of dimension r×k.

The approach is to fine-tune the matrix decomposition of ∆W i.e. matrices B and A, with a rank r significantly less than the `min(d, k)` from the `original matrix`. Hence, reducing the number of parameters we need to fine-tune,



That's a good understanding! You're absolutely right.

Matrix decomposition techniques like LoRA leverage the redundancy in the data (weights in this case) to achieve efficient fine-tuning. Here's a breakdown of the key points:

1. **Redundancy in Weights:** Large Language Models (LLMs) often learn weights that contain redundancies. This means some information might be captured in multiple weight values.
2. **Low-Rank Approximation:** Matrix decomposition techniques like LoRA can approximate the original weight matrix (delta(W)) with a lower-rank version. This essentially captures the important information in a smaller set of weights, eliminating redundancies.
3. **Lower Intrinsic Dimensionality:** The LoRA paper builds on the hypothesis that trained models have a low "intrinsic rank." This means the information needed to represent the model can be captured effectively in a lower dimensional space.
4. **Focusing on the Update:**  Instead of fine-tuning the entire weight matrix, LoRA focuses on the update (delta(W)) to the weights during adaptation. This update matrix is hypothesized to also have a low intrinsic rank, allowing for efficient fine-tuning using the lower-dimensional representation.

By exploiting these redundancies, LoRA achieves faster and more memory-efficient adaptation compared to fine-tuning the entire weight matrix. This makes it a valuable technique for adapting large models to new tasks.


Default LoRA with 16-bit brain floating point precision:

Training time: 1.85 h

Memory used: 21.33 GB

QLoRA with 4-bit Normal Floats:

Training time: 2.79 h

Memory used: 14.18 GB

You've got a clear understanding of the weight precision differences between LoRA and QLoRA! Here's a breakdown of the key points:

**Weight Precision:**

* **LoRA:** Uses 16-bit floating-point format (FP16) for tracked weights. This offers a good balance between precision and memory usage.
* **QLoRA:** Utilizes 4-bit quantization, significantly reducing memory footprint compared to LoRA.

**Benefits of QLoRA's Lower Precision:**

* **Reduced Memory Usage:**  QLoRA requires much less memory for weights, making it ideal for deployment on memory-constrained devices.
* **Faster Inference:**  The lower precision allows for faster computations during inference, leading to quicker model responses.
* **Improved Efficiency:**  QLoRA offers a significant advantage in terms of both memory usage and computational efficiency, making it more practical for real-world applications.

**Trade-off to Consider:**

It's important to remember that there's a trade-off between precision and efficiency. While QLoRA offers significant memory and speed benefits, the 4-bit quantization might introduce some accuracy loss compared to LoRA's 16-bit precision.

**Choosing Between LoRA and QLoRA:**

The choice between LoRA and QLoRA depends on your specific needs:

* If maintaining high accuracy is crucial, LoRA might be preferable.
* If memory and computational efficiency are top priorities, QLoRA could be the better option, especially if some accuracy loss is acceptable.

Overall, both LoRA and QLoRA are valuable techniques for efficient LLM fine-tuning. Understanding the weight precision difference and the associated trade-offs allows you to make an informed decision based on your specific application requirements.